<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/Freeze_Omni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/weedge/Freeze-Omni


Cloning into 'Freeze-Omni'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 114 (delta 31), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (114/114), 9.91 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
%cd Freeze-Omni


/content/Freeze-Omni


In [3]:
!pip install -q -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00


In [6]:
!pip install -q pyngrok==7.2.1

In [ ]:
!huggingface-cli download VITA-MLLM/Freeze-Omni --local-dir ./ckpts/VITA-MLLM/Freeze-Omni --local-dir-use-symlinks False


In [ ]:
!huggingface-cli download Qwen/Qwen2-7B-Instruct --local-dir ./ckpts/Qwen/Qwen2-7B-Instruct --local-dir-use-symlinks False


# inference

In [ ]:
!PYTHONPATH=./:$PYTHONPATH CUDA_VISIBLE_DEVICES=0 python bin/inference.py \
    --model_path ./ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --input_wav ./assets/hi.wav \
    --output_wav ./assets/answer_hi.wav \
    --llm_path ./ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8

Namespace(model_path='./ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='./ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, input_wav='./assets/hi.wav', output_wav='./assets/answer_hi.wav')
the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 10.90it/s]
Checkpoint: loading from checkpoint ./ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
/content/Freeze-Omni/models/utils.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

In [ ]:
from IPython.display import Audio
Audio(filename="./assets/hi.wav")


In [ ]:
from IPython.display import Audio
Audio(filename="./assets/answer_hi.wav")


# server

In [10]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
#HF_TOKEN=userdata.get('HF_TOKEN')


In [11]:
!ngrok config add-authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!cat /root/.config/ngrok/ngrok.yml

In [31]:
!PYTHONPATH=./:$PYTHONPATH PYTHONDONTWRITEBYTECODE=1 CUDA_VISIBLE_DEVICES=0 python bin/server.py --help

2024-12-15 10:13:58.073813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 10:13:58.090839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 10:13:58.111996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 10:13:58.118417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 10:13:58.133866: I tensorflow/core/platform/cpu_feature_guar

In [32]:
!PYTHONPATH=./:$PYTHONPATH PYTHONDONTWRITEBYTECODE=1 CUDA_VISIBLE_DEVICES=0 python bin/server.py \
    --ip 127.0.0.1 \
    --port 8000 \
    --max_users 3 \
    --llm_exec_nums 1 \
    --timeout 180 \
    --model_path ./ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --llm_path ./ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8 \
    --ngrok

2024-12-15 10:14:23.162370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 10:14:23.178708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-15 10:14:23.199675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 10:14:23.205984: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 10:14:23.220908: I tensorflow/core/platform/cpu_feature_guar